Importing Libraries

🚀 Steps to Train a Model in Amazon SageMaker
1. Import libraries
2. Create S3 bucket
3. Map Train and Train data in S3
4. Map model's path in S3

In [1]:
pip install --upgrade sagemaker


Note: you may need to restart the kernel to use updated packages.


In [2]:
import sagemaker 
import boto3   # To interact with AWS services 
from sagemaker.inputs import TrainingInput
from sagemaker.amazon.amazon_estimator import get_image_uri   ## Get image URI for XGBoost
from sagemaker.session import s3_input, Session   #connects your dataset stored in S3 to SageMaker training jobs, manages interactions between SageMaker and AWS resources.


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/02/25 18:37:04] INFO     Found credentials from IAM Role:                                   ]8;id=907389;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=849806;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
bucket_name = 'bankapplication13345' 
my_region = boto3.session.Session().region_name   # set the region of the instance
print(my_region)

us-east-1


In [4]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

[03/02/25 18:37:05] INFO     Found credentials from IAM Role:                                   ]8;id=707207;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=597605;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

S3 bucket created successfully


In [5]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication13345/xgboost-as-a-built-in-algo/output


Download and store the dataset in S3 bucket

In [6]:
import pandas as pd
import urllib 

try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank.csv")
    print('Success: downloaded bank.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
     

Success: downloaded bank.csv.
Success: Data loaded into dataframe.


In [7]:
print(model_data.head())

   age  campaign  pdays  previous  no_previous_contact  not_working  \
0   56         1    999         0                    1            0   
1   57         1    999         0                    1            0   
2   37         1    999         0                    1            0   
3   40         1    999         0                    1            0   
4   56         1    999         0                    1            0   

   job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  ...  \
0           0                0                 0              1  ...   
1           0                0                 0              0  ...   
2           0                0                 0              0  ...   
3           1                0                 0              0  ...   
4           0                0                 0              0  ...   

   day_of_week_fri  day_of_week_mon  day_of_week_thu  day_of_week_tue  \
0                0                1                0               

In [8]:
# Trian, Test, Split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729),[int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [9]:
### Saving Train And Test Into Buckets
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')    

[03/02/25 18:37:06] INFO     Found credentials from IAM Role:                                   ]8;id=369355;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=333207;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [10]:
import os
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], 
                                                axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')    

                    INFO     Found credentials from IAM Role:                                   ]8;id=641603;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=370614;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Build and Train Xgboost algo

In [11]:

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = sagemaker.image_uris.retrieve("xgboost", Session().boto_region_name, "1.7-1")
display(container)

[03/02/25 18:37:07] INFO     Ignoring unnecessary instance type: None.                            ]8;id=154840;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=129843;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'

In [22]:

# Define hyperparameters for XGBoost
hyperparameters = {
    "max_depth": 5,
    "eta": 0.2,
    "gamma": 4,
    "min_child_weight": 6,
    "subsample": 0.7,
    "verbosity": 0,
    "objective": "binary:logistic",
    "num_round": 50
}

# Define SageMaker Estimator
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)
print("Estimator successfully created!")


Estimator successfully created!


In [23]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

[03/02/25 18:48:45] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=446670;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=218808;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=12986;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=279223;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-03-02-18-48-45-296                                             

2025-03-02 18:48:46 Starting - Starting the training job...
..25-03-02 18:49:19 Downloading - Downloading input data.
..25-03-02 18:49:34 Downloading - Downloading the training image.
..25-03-02 18:50:15 Training - Training image download completed. Training in progress..
2025-03-02 18:50:48 Uploading - Uploading generated training model
2025-03-02 18:50:48 Completed - Training job completed
[2025-03-02 18:50:33.184 ip-10-0-145-220.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-03-02 18:50:33.206 ip-10-0-145-220.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-03-02:18:50:33:INFO] Imported framework sagemaker_xgboost_container.training
[2025-03-02:18:50:33:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2025-03-02:18:50:33:INFO] No GPUs detected (normal if no gpus installed)
[2025-03-02:18:50:33:INFO] Running XGBoost Sagemaker in algorithm mode
[2025-03-02:18:50:33:

In [24]:
print("Latest Training Job Name:", estimator.latest_training_job)


Latest Training Job Name: <sagemaker.estimator._TrainingJob object at 0x7fa0d9931720>


In [27]:
xgb_predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.2xlarge")


[03/02/25 18:53:21] INFO     Creating model with name: sagemaker-xgboost-2025-03-02-18-53-21-088    ]8;id=729820;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=532982;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=253827;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=228394;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             sagemaker-xgboost-2025-03-02-18-53-21-088                                             

                    INFO     Creating endpoint with name sagemaker-xgboost-2025-03-02-18-53-21-088  ]8;id=961511;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=79038;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!

In [37]:
import numpy as np
from sagemaker.serializers import CSVSerializer  
# Convert test data to a NumPy array (drop target columns)
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Configure SageMaker Predictor
xgb_predictor.content_type = "text/csv"
xgb_predictor.serializer = CSVSerializer() 
# Make Predictions for Each Row Separately (Ensures Correct Shape)
predictions = []
for row in test_data_array:
    pred = xgb_predictor.predict(row.reshape(1, -1)).decode("utf-8")  
    predictions.append(float(pred.strip()))  
# Convert to NumPy array
predictions_array = np.array(predictions)

# Print the correct shape
print("Predictions Shape:", predictions_array.shape)


Predictions Shape: (12357,)


In [38]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

$500 * FN(C) + $0 * TN(C) + $100 * FP(C) + $100 * TP(C)

In [49]:
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
     


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [50]:

sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[03/02/25 19:22:42] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=558593;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=733669;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Deleting endpoint with name: sagemaker-xgboost-2025-03-02-18-53-21-088 ]8;id=936293;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=587027;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\

[{'ResponseMetadata': {'RequestId': 'S4W58KZWJPXCBRBC',
   'HostId': 'gB3hgASKdJFoA7GQTZdb7rNL/qb99XUXEw2icB6Nprb/HwRjxfau8KGyYMnRYEnJ/F76lifFSJ74fifbMlfhxgmtZNpq3H8YWdlRbLhrP0M=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'gB3hgASKdJFoA7GQTZdb7rNL/qb99XUXEw2icB6Nprb/HwRjxfau8KGyYMnRYEnJ/F76lifFSJ74fifbMlfhxgmtZNpq3H8YWdlRbLhrP0M=',
    'x-amz-request-id': 'S4W58KZWJPXCBRBC',
    'date': 'Sun, 02 Mar 2025 19:22:43 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-03-02-18-35-28-563/debug-output/index/000000000/000000000020_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-03-02-18-35-28-563/debug-output/events/000000000040/000000000040_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-03-02-18-40-27-678/debug